# Importando bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error, r2_score

# Puxando data frame

Rodando notebook de pré processamento dos dados.

In [ ]:
%run Pre_Processamento.ipynb

Recuperando o data frame pré processado.

In [ ]:
%store -r df_1

Separando o dataset entre dados de teste e de treino com o método split, que realiza de forma aleatória ao longo do dataframe.

In [ ]:
X = df.drop(['items_sold','revenue'], axis=1) 
y = df['items_sold']

SEED = 42

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=SEED)


## Seleção de Features

Aplicamos a seleção de features em nosso projeto para identificar as variáveis mais impactantes para a previsão de "items_sold", reduzindo assim a complexidade do modelo e o tempo de treinamento. Isso nos permite focar nas features que realmente importam, evitando o ruído e o overfitting, o que contribui para um modelo mais preciso e eficiente.

### Feature Importances do Random Forest

O método Feature Importances usa o algoritmo Random Forest para mostrar quais variáveis têm mais impacto nas previsões. No entanto, uma observação é que esse método nos faz escolher previamente o número de variáveis importantes que queremos focar, o que pode não ser o mais ideal. Nesse caso, decidimos analisar as 5 melhores features com o método. (explicar porquê 5)

In [ ]:
rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)

In [ ]:
importances = rf_model.feature_importances_
indices = np.argsort(importances)[-5:]
X_train_rf_selected = X_train.iloc[:, indices]
X_test_rf_selected = X_test.iloc[:, indices]

In [ ]:
X_train_rf_selected.head(5)

### Select KBest

O método Select KBest utiliza testes estatísticos para identificar as variáveis que têm maior influência no resultado que queremos prever. Similar ao método de Feature Importances, ele também requer que definamos antecipadamente o número de variáveis importantes a considerar. Aqui também optasmos por analisar as 5 melhores features através deste método.

In [ ]:
kbest = SelectKBest(score_func=f_classif, k=5)
X_train_kbest_selected = kbest.fit_transform(X_train, y_train)
X_test_kbest_selected = kbest.transform(X_test)
selected_cols = X_train.columns[kbest.get_support()]

In [ ]:
selected_cols

### Matriz de correlação em heatmap

A matriz de correlação é uma ferramenta essencial na análise de dados que descreve as relações entre as variáveis de um conjunto de dados. No contexto da aplicação do nosso modelo, a matriz de correlação desempenha um papel crucial na compreensão das interações entre as diferentes variáveis que podem influenciar as vendas na Mobly, de modo a trazer insights sobre quais variáveis priorizar ou aquelas que não impactam de forma significativa.

Essa matriz nos fornece uma visão geral das associações entre as variáveis, indicando se elas se movem juntas ou em direções opostas. Para construir a matriz de correlação, calculamos coeficientes de correlação, como o coeficiente de correlação de Pearson, entre pares de variáveis. Esses coeficientes variam de -1 a 1, onde -1 representa uma correlação negativa perfeita, 1 representa uma correlação positiva perfeita e 0 indica ausência de correlação.

Ao examinar a matriz de correlação, podemos identificar variáveis que têm forte influência nas vendas da Mobly e, assim, priorizar nossos esforços na modelagem. Essa análise também pode nos alertar para possíveis multicolinearidades, onde duas variáveis estão altamente correlacionadas entre si, o que pode afetar a estabilidade do modelo.

In [ ]:
corr_matrix = df.corr()


labels = ['sku', 'unit_price', 'anchor_category', 'product_department',
       'product_category', 'sku_color', 'winning_price',
       'avg_website_visits_last_week', 'stock_qty', 'items_sold_b"undle',
       'dolar', 'shipment_type_próprio', 'origin_country_Nacional',
       'price_status_Ganhando', 'price_status_Perdendo',
       'price_status_Sem Registro', 'price_status_Único Disponível',
       'process_costing_yes', 'mes', 'dia', 'semana_do_ano', 'trimestre',
       'dia_do_ano', 'dia_da_semana', 'fim_de_semana', 'dia_do_ano_sin',
       'dia_do_ano_cos', 'dia_da_semana_sin', 'dia_da_semana_cos', 'ano_2021',
       'ano_2022', 'ano_2023']
sns.heatmap(
    corr_matrix,
    xticklabels=labels,
    yticklabels=labels,
)

Em conclusão, foi possível perceber diversasa análises com grande contribuição para o projeto. Por exemplo, houve uma altissima correlação entre winning_price e process_costing yes, o que era esperado, uma vez que produtos beneficiados tem um preço menor, e portanto, maior vendas. Foi identificado também uma inesperada correlação entre o dólar, a média de visita do site e a quantidade de estoque, estando com uma correlação maior que o esperado, indicando a relevâncias desses atributos no modelo. Portanto, a matriz de correlação foi uma ferramenta essencial na etapa de preparação de dados e análise exploratória, ajudando-nos a entender as relações entre as variáveis e a escolher as mais relevantes para a previsão de vendas na Mobly, contribuindo para um modelo mais preciso e eficaz.